#06-Tipagem de dados com SQL e Pyspark
by Adilson Albino

In [0]:
#Importando funções sql
from pyspark.sql.functions import *

In [0]:
df_carros = spark\
.read.format('csv')\
.option('header', True)\
.load('/aprendizado/modelo_carro')

df_carros.show(10)

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
|       4|          EX|$84981.12|       23|
|       5|      Escort|$77466.89|       17|
|       6|  Expedition|$84698.71|       17|
|       7|     Voyager|$95567.75|       42|
|       8|       Civic|$84749.22|       20|
|       9|    Defender|$98600.79|       29|
|      10|V8 Vantage S|$94791.61|        2|
+--------+------------+---------+---------+
only showing top 10 rows



In [0]:
#substituindo $ por nada.
df_carros = df_carros.withColumn("preco", regexp_replace(col("preco"),"\$", ""))
df_carros.show(10)

+--------+------------+--------+---------+
|id_carro|modelo_carro|   preco|cod_marca|
+--------+------------+--------+---------+
|       1|      Avalon|78401.95|       54|
|       2|         RDX|95987.38|        1|
|       3|        Golf|61274.55|       55|
|       4|          EX|84981.12|       23|
|       5|      Escort|77466.89|       17|
|       6|  Expedition|84698.71|       17|
|       7|     Voyager|95567.75|       42|
|       8|       Civic|84749.22|       20|
|       9|    Defender|98600.79|       29|
|      10|V8 Vantage S|94791.61|        2|
+--------+------------+--------+---------+
only showing top 10 rows



In [0]:
#Vamos usar o camando CAST para alterar a tipagem das colunas do dataframe
#Os dados da tabela estão em string as colunas
#_sqldf:pyspark.sql.dataframe.DataFrame
#id_carro:string
#modelo_carro:string
#preco:string
#cod_marca:string

#Com o comando CAST podemos alterar essa essa tipagem.

#Utilizando SQL

In [0]:
#Criando uma tabela temporária para consultas sql
df_carros.createOrReplaceTempView("carros")

#criando dataframe para armazenar dados de retorno do comando sql
df_carros_sql = spark.sql("""
 select
cast(id_carro as int),
modelo_carro,
cast(preco as double),
cast(cod_marca as long)
from carros                         
                          
""")

df_carros_sql.show(10)

+--------+------------+--------+---------+
|id_carro|modelo_carro|   preco|cod_marca|
+--------+------------+--------+---------+
|       1|      Avalon|78401.95|       54|
|       2|         RDX|95987.38|        1|
|       3|        Golf|61274.55|       55|
|       4|          EX|84981.12|       23|
|       5|      Escort|77466.89|       17|
|       6|  Expedition|84698.71|       17|
|       7|     Voyager|95567.75|       42|
|       8|       Civic|84749.22|       20|
|       9|    Defender|98600.79|       29|
|      10|V8 Vantage S|94791.61|        2|
+--------+------------+--------+---------+
only showing top 10 rows



In [0]:
#Como checar de outra forma os tipos de dados?
#No pyspark podemos utiliza o "printSchema"

df_carros_sql.printSchema()

root
 |-- id_carro: integer (nullable = true)
 |-- modelo_carro: string (nullable = true)
 |-- preco: double (nullable = true)
 |-- cod_marca: long (nullable = true)



#Utilizando PYSPARK

In [0]:
df_carros_spark = df_carros
df_carros_spark.printSchema()

root
 |-- id_carro: string (nullable = true)
 |-- modelo_carro: string (nullable = true)
 |-- preco: string (nullable = true)
 |-- cod_marca: string (nullable = true)



In [0]:
df_carros_spark = df_carros_spark.withColumn(
    "id_carro",
    col("id_carro").cast("int")
).withColumn(
    "preco",
    col("preco").cast("double")
).withColumn(
    "cod_marca",
   col("cod_marca").cast("long")
)
df_carros_spark.printSchema()

root
 |-- id_carro: integer (nullable = true)
 |-- modelo_carro: string (nullable = true)
 |-- preco: double (nullable = true)
 |-- cod_marca: long (nullable = true)



In [0]:
#Uma segunda forma de se fazer com o pyspark é usando o .SELECT

df_carros_spark2 = df_carros
df_carros_spark2 = df_carros_spark2.select(
    col("id_carro").cast("int"),
    col("preco").cast("double"),
    col("cod_marca").cast("long"),
    "modelo_carro"
)
df_carros_spark2.printSchema()
df_carros_spark2.show(10)

root
 |-- id_carro: integer (nullable = true)
 |-- preco: double (nullable = true)
 |-- cod_marca: long (nullable = true)
 |-- modelo_carro: string (nullable = true)

+--------+--------+---------+------------+
|id_carro|   preco|cod_marca|modelo_carro|
+--------+--------+---------+------------+
|       1|78401.95|       54|      Avalon|
|       2|95987.38|        1|         RDX|
|       3|61274.55|       55|        Golf|
|       4|84981.12|       23|          EX|
|       5|77466.89|       17|      Escort|
|       6|84698.71|       17|  Expedition|
|       7|95567.75|       42|     Voyager|
|       8|84749.22|       20|       Civic|
|       9|98600.79|       29|    Defender|
|      10|94791.61|        2|V8 Vantage S|
+--------+--------+---------+------------+
only showing top 10 rows



In [0]:
#Uma terceira forma de se fazer com o pyspark é usando funções de tipagem dentro do CAST

from pyspark.sql.types import *

df_carros_spark2 = df_carros
df_carros_spark2 = df_carros_spark2.select(
    col("id_carro").cast(IntegerType()),
    col("preco").cast(DoubleType()),
    col("cod_marca").cast(LongType()),
    "modelo_carro"
)
df_carros_spark2.printSchema()
df_carros_spark2.show(10)

root
 |-- id_carro: integer (nullable = true)
 |-- preco: double (nullable = true)
 |-- cod_marca: long (nullable = true)
 |-- modelo_carro: string (nullable = true)

+--------+--------+---------+------------+
|id_carro|   preco|cod_marca|modelo_carro|
+--------+--------+---------+------------+
|       1|78401.95|       54|      Avalon|
|       2|95987.38|        1|         RDX|
|       3|61274.55|       55|        Golf|
|       4|84981.12|       23|          EX|
|       5|77466.89|       17|      Escort|
|       6|84698.71|       17|  Expedition|
|       7|95567.75|       42|     Voyager|
|       8|84749.22|       20|       Civic|
|       9|98600.79|       29|    Defender|
|      10|94791.61|        2|V8 Vantage S|
+--------+--------+---------+------------+
only showing top 10 rows

